# Конвертация YOLOv8 модели в формат RKNN

## dynamic=False
> Определяет, будет ли размер входа динамическим. Значение: Почему важно: Для RKNN обязательно нужно dynamic=False, потому что NPU плохо работает с переменными размерами.
- False: вход фиксированного размера (например, [1, 3, 640, 640]).
- True: вход может быть переменным (например, [1, 3, H, W]).

## simplify=True
> Упрощает ONNX-граф (удаляет лишние операции, объединяет блоки). Всегда ставить True, особенно перед импортом в RKNN Toolkit.
- Повышает производительность.
- Снижает размер файла.
- Устраняет потенциальные ошибки при конвертации.

## opset=12
> Указывает версию ONNX-операторов (opset). Некоторые версии RKNN Toolkit поддерживают только определённые версии opset. Версия 12 — стабильная и хорошо поддерживается большинством инструментов, включая RKNN Toolkit 1.x и 2.x.

In [1]:
from ultralytics import YOLO

In [3]:
model = YOLO('../Weight/best.pt')
model.export(format="onnx", dynamic=True, simplify=True, opset=11)
# nms=False

Ultralytics 8.3.138 🚀 Python-3.10.12 torch-2.2.0 CPU (Cortex-A55)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs



PyTorch: starting from '../Weight/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (21.5 MB)

ONNX: starting export with onnx 1.16.1 opset 11...
ONNX: slimming with onnxslim 0.1.53...
ONNX: export success ✅ 9.0s, saved as '../Weight/best.onnx' (42.5 MB)

Export complete (13.6s)
Results saved to /home/orangepi/Desktop/DetectPeople/Weight
Predict:         yolo predict task=detect model=../Weight/best.onnx imgsz=640  
Validate:        yolo val task=detect model=../Weight/best.onnx imgsz=640 data=./DataSet/DataFullHD/Split/dataset.yaml  
Visualize:       https://netron.app


'../Weight/best.onnx'

# Скрипт для конвертации ONNX → RKNN

In [4]:
from rknn.api import RKNN

ONNX_MODEL = '../Weight/best.onnx'
RKNN_MODEL = '../Weight/best.rknn'
RKNN_MODEL_TXT = '/home/orangepi/Desktop/DetectPeople/Data/dataSet/dataset.txt'

rknn = RKNN()

I rknn-toolkit2 version: 2.3.2


In [5]:
print('Конфигурирование модели...')
ret = rknn.config(
    mean_values=[[0, 0, 0]], 
    std_values=[[255, 255, 255]], 
    target_platform='rk3588',  # укажи тут свою платформу, например rk3566, rk3568 и т.д.
    dynamic_input=[
        [
            [1, 3, 640, 640],
        ]
    ],
)
if ret != 0:
    print('Ошибка конфигурации модели')
    exit(ret)

W config: Please make sure the model can be dynamic when enable 'config.dynamic_input'!
I The 'dynamic_input' function has been enabled, the MaxShape is dynamic_input[0] = [[1, 3, 640, 640]]!
          The following functions are subject to the MaxShape:
            1. The quantified dataset needs to be configured according to MaxShape
            2. The eval_perf or eval_memory return the results of MaxShape


Конфигурирование модели...


In [6]:
print('Загрузка ONNX модели...')
ret = rknn.load_onnx(model=ONNX_MODEL)
if ret != 0:
    print('Ошибка загрузки ONNX модели')
    exit(ret)

Загрузка ONNX модели...


I Loading : 100%|██████████████████████████████████████████████| 150/150 [00:00<00:00, 12884.14it/s]


In [7]:
print('Компиляция модели...')
ret = rknn.build(do_quantization=False)
if ret != 0:
    print('Ошибка компиляции модели')
    exit(ret)

    # ret = rknn.build(do_quantization=True, dataset=RKNN_MODEL_TXT)

Компиляция модели...


I OpFusing 1 :   0%|                                                        | 0/100 [00:00<?, ?it/s]W build: The 'dynamic_input' function is enabled, disable _p_convert_maxpool_to_maxpool_tile!
W build: The 'dynamic_input' function is enabled, disable _p_convert_maxpool_to_maxpool_tile!
W build: The 'dynamic_input' function is enabled, disable _p_convert_maxpool_to_maxpool_tile!
I OpFusing 1 :  98%|█████████████████████████████████████████████ | 98/100 [00:00<00:00, 160.36it/s]

W build: The 'dynamic_input' function is enabled, disable _p_convert_maxpool_to_maxpool_tile!
W build: The 'dynamic_input' function is enabled, disable _p_convert_maxpool_to_maxpool_tile!
W build: The 'dynamic_input' function is enabled, disable _p_convert_maxpool_to_maxpool_tile!
I OpFusing 1 :  95%|███████████████████████████████████████████▋  | 95/100 [00:00<00:00, 118.61it/s]W build: The 'dynamic_input' function is enabled, disable _p_convert_maxpool_to_maxpool_tile!
W build: The 'dynamic_input' function is enabled, disable _p_convert_maxpool_to_maxpool_tile!
W build: The 'dynamic_input' function is enabled, disable _p_convert_maxpool_to_maxpool_tile!
I OpFusing 1 :  92%|███████████████████████████████████████████▏   | 92/100 [00:00<00:00, 93.50it/s]W build: The 'dynamic_input' function is enabled, disable _p_convert_maxpool_to_maxpool_tile!
W build: The 'dynamic_input' function is enabled, disable _p_convert_maxpool_to_maxpool_tile!
W build: The 'dynamic_input' function is enabled

I OpFusing 1 :   0%|                                                        | 0/100 [00:01<?, ?it/s]W build: The 'dynamic_input' function is enabled, disable _p_convert_maxpool_to_maxpool_tile!
W build: The 'dynamic_input' function is enabled, disable _p_convert_maxpool_to_maxpool_tile!
W build: The 'dynamic_input' function is enabled, disable _p_convert_maxpool_to_maxpool_tile!
I OpFusing 1 :  93%|███████████████████████████████████████████▋   | 93/100 [00:01<00:00, 54.69it/s]W build: The 'dynamic_input' function is enabled, disable _p_convert_maxpool_to_maxpool_tile!
W build: The 'dynamic_input' function is enabled, disable _p_convert_maxpool_to_maxpool_tile!
W build: The 'dynamic_input' function is enabled, disable _p_convert_maxpool_to_maxpool_tile!
I OpFusing 1 : 100%|██████████████████████████████████████████████| 100/100 [00:01<00:00, 53.33it/s]

I OpFusing 1 :   0%|                                                        | 0/100 [00:01<?, ?it/s]W build: The 'dynamic_input' function is enabled, disable _p_convert_maxpool_to_maxpool_tile!
W build: The 'dynamic_input' function is enabled, disable _p_convert_maxpool_to_maxpool_tile!
W build: The 'dynamic_input' function is enabled, disable _p_convert_maxpool_to_maxpool_tile!
I OpFusing 2 : 100%|██████████████████████████████████████████████| 100/100 [00:02<00:00, 48.06it/s]


I OpFusing 2 : 100%|████████████████████████████████████████████| 100/100 [00:00<00:00, 1345.97it/s]
I rknn building ...


I rknn building done.


In [8]:
print('Экспорт RKNN модели...')
ret = rknn.export_rknn(RKNN_MODEL)
if ret != 0:
    print('Ошибка экспорта RKNN модели')
    exit(ret)

print('Готово!')

Экспорт RKNN модели...
Готово!
